## Part 4: Fraud

In [1]:
%run "../scripts/outlier.py" "../scripts/paths.json"

22/10/06 12:37:52 WARN Utils: Your hostname, mast30034 resolves to a loopback address: 127.0.1.1; using 45.113.234.45 instead (on interface eth0)
22/10/06 12:37:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 12:37:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/06 12:37:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/06 12:37:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


22/10/06 12:38:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [39]:
%run "../scripts/fraud_feature.py" "../scripts/paths.json"

In [3]:
#Loading consumer fraud data: 
fraud_consumer = pd.read_csv('../data/tables/consumer_fraud_probability.csv')
fraud_consumer = spark.createDataFrame(fraud_consumer) 

#Loading merchant fraud data: 
fraud_merchant = pd.read_csv('../data/tables/merchant_fraud_probability.csv')
fraud_merchant = spark.createDataFrame(fraud_merchant) 


In [4]:
full_data = spark.read.parquet('../data/tables/full_join.parquet')

In [5]:
full_data.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,postcode,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id
Nullam Nisl Insti...,15912561882,shoe shops,3.72,b,Allison Stevens,60866 Miller Avenue,NSW,1193,Female,15912561882,28.083206954036427,bf851e23-d713-441...,2021-09-06,6305,6834
Nullam Nisl Insti...,15912561882,shoe shops,3.72,b,Allison Stevens,60866 Miller Avenue,NSW,1193,Female,15912561882,13.671791104152105,0cad39e4-71f7-47c...,2021-11-27,6305,6834
Nullam Nisl Insti...,15912561882,shoe shops,3.72,b,Lawrence Tyler,3511 Victoria Meadow,NSW,2212,Male,15912561882,235.75679877020391,3fdd2acd-3b77-4c2...,2021-06-18,18983,36103
Nullam Nisl Insti...,15912561882,shoe shops,3.72,b,Ryan Crawford,7107 Michelle Har...,NSW,1126,Male,15912561882,284.642162569933,f8303e6b-78f6-481...,2021-04-29,12285,37894
Nullam Nisl Insti...,15912561882,shoe shops,3.72,b,Alexander Skinner,35694 Kimberly Co...,VIC,3053,Male,15912561882,213.64355235308224,4b193807-a752-4f6...,2021-07-26,9248,147322


The consumer fraud data shows the list of transactions with a high probability of being fraudulent, and therefore should be considered invalid.

In [6]:
print(fraud_consumer.count())
fraud_consumer = fraud_consumer.withColumnRenamed("fraud_probability","cons_fraud_prob").withColumnRenamed("order_datetime","order_datetime_1").withColumnRenamed("user_id","user_id_1")
fraud_consumer.limit(5)

34864


user_id_1,order_datetime_1,cons_fraud_prob
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777
3101,2021-04-17,91.42192091901347
22239,2021-10-19,94.70342477508036


Probability of fraud for high revenue merchants...?

In [7]:
print(fraud_merchant.count())
fraud_merchant = fraud_merchant.withColumnRenamed("merchant_abn","merchant_abn_1").withColumnRenamed("fraud_probability","merch_fraud_prob")
fraud_merchant.limit(5)

114


merchant_abn_1,order_datetime,merch_fraud_prob
19492220327,2021-11-28,44.40365864749536
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902


In [8]:

merchant_data = full_data.select("merchant_abn", "categories","revenue_levels","dollar_value","user_id","consumer_id")

In [9]:
merchant_data.limit(5)

merchant_abn,categories,revenue_levels,dollar_value,user_id,consumer_id
15912561882,shoe shops,b,28.083206954036427,6305,6834
15912561882,shoe shops,b,13.671791104152105,6305,6834
15912561882,shoe shops,b,235.75679877020391,18983,36103
15912561882,shoe shops,b,284.642162569933,12285,37894
15912561882,shoe shops,b,213.64355235308224,9248,147322


- join consumer fraud to final join
- create a version of final join with removed transactions from fraud
- aggregate final join from transaction to merchant level
- add fraud merchant as a feature to aggregated final join
- treat fraud as response and try to create model to give other merchants a fraud score

In [10]:
#Joining consumer fraud, merchant data, and merchant fraud data to one datatset: 

#Joining consumer fraud and merchant data together

merchant_data.createOrReplaceTempView("temp")

fraud_consumer.createOrReplaceTempView("temp2")

join_1 = spark.sql(""" 

SELECT *
FROM temp

LEFT JOIN temp2

ON temp.user_id = temp2.user_id_1 
""")


In [11]:
#Joining join_1 to merchant fraud 

#Joining all test + predicting Sets Together 
join_1.createOrReplaceTempView("temp")

fraud_merchant.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2.merchant_abn_1
""")

In [12]:
# #Changing all null probability values to 0.01: 

# MIN_P = 0.01
# final_join = final_join.fillna(MIN_P, subset=["cons_fraud_prob", "merch_fraud_prob"])

In [13]:
#join segment data 
#clustering 
#decide threshold based on clusters 
#use threshold to classify if transaction is fradulent or not 

In [14]:
# Reading in segments 
segments_data = spark.read.csv('../data/curated/tagged_merchants.csv')

In [15]:
segments_data.limit(5)

_c0,_c1,_c2,_c3,_c4,_c5,_c6
null,name,tags,merchant_abn,cleaned_tags,store_type,category
0,Felis Limited,"((furniture, home...",10023283211,furniture home fu...,0,Furniture
1,Arcu Ac Orci Corp...,"([cable, satellit...",10142254217,cable satellite a...,4,Electronics
2,Nunc Sed Company,"([jewelry, watch,...",10165489824,jewelry watch clo...,1,Toys and DIY
3,Ultricies Digniss...,"([wAtch, clock, a...",10187291046,watch clock and j...,1,Toys and DIY


In [16]:
#Removing false header line 
segments_data.na.drop(subset=["_c0"])

_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,Felis Limited,"((furniture, home...",10023283211,furniture home fu...,0,Furniture
1,Arcu Ac Orci Corp...,"([cable, satellit...",10142254217,cable satellite a...,4,Electronics
2,Nunc Sed Company,"([jewelry, watch,...",10165489824,jewelry watch clo...,1,Toys and DIY
3,Ultricies Digniss...,"([wAtch, clock, a...",10187291046,watch clock and j...,1,Toys and DIY
4,Enim Condimentum PC,([music shops - m...,10192359162,music shops musi...,3,"Books, Stationary..."
5,Fusce Company,"[(gift, card, nov...",10206519221,gift card novelty...,4,Electronics
6,Aliquam Enim Inco...,"[(computers, comP...",10255988167,computers compute...,4,Electronics
7,Ipsum Primis Ltd,"[[watch, clock, a...",10264435225,watch clock and j...,1,Toys and DIY
8,Pede Ultrices Ind...,([computer progra...,10279061213,computer programm...,2,"Beauty, Health, P..."
9,Nunc Inc.,"[(furniture, home...",10323485998,furniture home fu...,0,Furniture


In [17]:
final_join.createOrReplaceTempView("temp")

segments_data.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2._c3
""")

In [18]:
# final_join.limit(5)
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import when
from pyspark.sql import functions as countDistinct

In [19]:
#Creating a table of average fraud probabilities for each category of purchase: 

aggregated_prob = final_join.groupBy("temp2._c6").agg(F.avg("cons_fraud_prob").\
alias("avg_consumer_fraud"),F.avg("merch_fraud_prob").alias("avg_merchant_fraud"))

In [20]:
aggregated_prob.show()

+--------------------+------------------+------------------+
|                 _c6|avg_consumer_fraud|avg_merchant_fraud|
+--------------------+------------------+------------------+
|                null| 15.14504487064014|              null|
|         Electronics|15.099318215130051|29.635051745296614|
|        Toys and DIY| 15.13486107948872| 32.39127563989597|
|           Furniture| 15.10712522737528| 30.93576502384926|
|Beauty, Health, P...|15.156273881317677|29.761851644961112|
|Books, Stationary...| 15.11163595522947| 29.01663781266414|
+--------------------+------------------+------------------+



In [21]:
#adding a column for is_fraud for each transaction: 

is_fraud = when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085)&(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Toys and DIY") & (final_join["cons_fraud_prob"] > 15.843) &(final_join["merch_fraud_prob"]>32.404),1)\
.when((final_join["_c6"] == "Furniture") & (final_join["cons_fraud_prob"] > 15.077) & (final_join["merch_fraud_prob"]>30.941),1)\
.when((final_join["_c6"] == "Beauty, Health, Personal and Household") & (final_join["cons_fraud_prob"] > 15.392) & (final_join["merch_fraud_prob"]>29.76),1)\
.when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085) &(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Books, Stationary and Music") & (final_join["cons_fraud_prob"] > 15.140) &(final_join["merch_fraud_prob"]>29.016),1).otherwise(0)

In [22]:
aggregated_prob.limit(6)

_c6,avg_consumer_fraud,avg_merchant_fraud
null,15.14504487064014,null
Electronics,15.099318215130051,29.635051745296614
Toys and DIY,15.13486107948872,32.39127563989597
Furniture,15.10712522737528,30.93576502384926
"Beauty, Health, P...",15.156273881317677,29.761851644961112
"Books, Stationary...",15.11163595522947,29.01663781266414


In [23]:
final_join.limit(5)

merchant_abn,categories,revenue_levels,dollar_value,user_id,consumer_id,user_id_1,order_datetime_1,cons_fraud_prob,merchant_abn_1,order_datetime,merch_fraud_prob,_c0,_c1,_c2,_c3,_c4,_c5,_c6
15912561882,shoe shops,b,284.642162569933,12285,37894,12285,2021-04-07,17.34081938676617,null,null,null,269,Nullam Nisl Insti...,"[[shoe shops], [b...",15912561882,shoe shops,0,Furniture
15912561882,shoe shops,b,28.083206954036427,6305,6834,6305,2021-09-14,8.999053519584878,null,null,null,269,Nullam Nisl Insti...,"[[shoe shops], [b...",15912561882,shoe shops,0,Furniture
15912561882,shoe shops,b,28.083206954036427,6305,6834,6305,2021-04-27,56.67489226765316,null,null,null,269,Nullam Nisl Insti...,"[[shoe shops], [b...",15912561882,shoe shops,0,Furniture
15912561882,shoe shops,b,13.671791104152105,6305,6834,6305,2021-09-14,8.999053519584878,null,null,null,269,Nullam Nisl Insti...,"[[shoe shops], [b...",15912561882,shoe shops,0,Furniture
15912561882,shoe shops,b,13.671791104152105,6305,6834,6305,2021-04-27,56.67489226765316,null,null,null,269,Nullam Nisl Insti...,"[[shoe shops], [b...",15912561882,shoe shops,0,Furniture


In [24]:
model_with_fraud = final_join.withColumn("is_fraud",is_fraud)
model_with_fraud = model_with_fraud.withColumnRenamed("merchant_abn","sus_merchant_abn")

In [25]:
# # model_with_fraud.count()
# merchant_abn_col = model_with_fraud.select("merchant_abn")
# print(merchant_abn_col.count())

In [26]:
# model_with_fraud.write.parquet("../data/curated/model_data_with_fraud_col.parquet") 

In [35]:
#Creating a table of average fraud probabilities for each category of purchase: 
final_model_with_fraud = model_with_fraud.groupBy("sus_merchant_abn").agg(F.avg("is_fraud").alias("average_fraud_rate_per_merchant"))


In [36]:

final_model_with_fraud.count()

4027

In [37]:

final_model_with_fraud.filter(final_model_with_fraud.average_fraud_rate_per_merchant != 0.0).show(5)

+----------------+-------------------------------+
|sus_merchant_abn|average_fraud_rate_per_merchant|
+----------------+-------------------------------+
|     90568944804|            0.16653537563822027|
|     27093785141|            0.17279945567320953|
|     76968105359|             0.2506265664160401|
|     19492220327|            0.15399746192893402|
|     83199298021|            0.08577235772357723|
+----------------+-------------------------------+
only showing top 5 rows



In [30]:
final_model_with_fraud.write.mode('overwrite').parquet("../data/tables/transactions_with_fraud_rates.parquet")

In [31]:
#merging with full dataset with all columns for geospatial visualisations 
full_data
final_model_with_fraud

full_data.createOrReplaceTempView("temp")

final_model_with_fraud.createOrReplaceTempView("temp2")

full_data_with_fraud = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2.sus_merchant_abn
""")

full_data_with_fraud.write.mode('overwrite').parquet('../data/tables/full_dataset_with_fraud_rates.parquet')



In [33]:
full_data_with_fraud.tail(10)

[Row(merchant_name='Nunc Risus LLP', merchant_abn=99009287608, categories='computer programming , data processing, and integrated systems design services', take_rate=3.15, revenue_levels='b', name='Holly Williams', address='37756 Christopher Orchard', state='VIC', postcode='3022', gender='Female', trans_merchant_abn=99009287608, dollar_value=48.66654072282729, order_id='046f358f-73a5-481b-a571-b7bbb60d9dbb', order_datetime=datetime.date(2022, 2, 23), user_id=8197, consumer_id=1474529, sus_merchant_abn=99009287608, average fraud rate per merchant=0.0),
 Row(merchant_name='Nunc Risus LLP', merchant_abn=99009287608, categories='computer programming , data processing, and integrated systems design services', take_rate=3.15, revenue_levels='b', name='Daniel Benton', address='452 Harrison Estate', state='QLD', postcode='4852', gender='Male', trans_merchant_abn=99009287608, dollar_value=371.3071592537628, order_id='3b95251e-2038-40a5-8df0-bdd0d1824cd7', order_datetime=datetime.date(2021, 12, 